In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

from torch.distributions.multivariate_normal import MultivariateNormal

In [3]:
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
length = 10

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        self.mu1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.mu2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.std1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        self.std2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        

        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0
        std1=torch.exp(self.std1).clip(1e-16,1)
        std2=torch.exp(self.std2).clip(1e-16,1)
        

        x_temp=x.clone()
            
            
            
            
        dist=MultivariateNormal(self.mu1[0]+self.mu2[0]*0.0, (std1[0])*torch.eye(length).to(device))
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        dist=MultivariateNormal(self.mu1[1]+self.mu2[1]*0.0,( std1[1])*torch.eye(length).to(device))

        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        dist=MultivariateNormal(self.mu1[2]+self.mu2[2]*0.0, (std1[2])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        dist=MultivariateNormal(self.mu1[3]+self.mu2[3]*0.0, (std1[3])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-16,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            dist=MultivariateNormal(self.mu1[0]+self.mu2[0]*i, (std1[0])*torch.eye(length).to(device))
        
            o_1 = torch.exp(dist.log_prob(pred[i]))

            x_temp[0]=o_1*x[0].clone()

            dist=MultivariateNormal(self.mu1[1]+self.mu2[1]*i, (std1[1])*torch.eye(length).to(device))

            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1].clone()

            dist=MultivariateNormal(self.mu1[2]+self.mu2[2]*i,( std1[2])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2].clone()

            dist=MultivariateNormal(self.mu1[3]+self.mu2[3]*i, (std1[3])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()

            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-16,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x
    def Observation(self,x):

#         o_1= self.emission((x))
#         o_1[1]=torch.exp(o_1[1])

        return mu
    
    def simulate(self):
         o=[]
         x1 = 0
         for i in range(0,1000):
             print(x1)
            
            
            
            
             if x1==0:
                x=torch.tensor([1.0,0.0,0]).to(device)
             if x1==1:
                x=torch.tensor([0, 1.0,0.0]).to(device)
             if x1==2:
                x=torch.tensor([0.0,0.0,1]).to(device)
             o_1= self.emission((x)) 
             o_1=o_1/torch.sum(o_1)

             o.append(torch.distributions.Categorical(o_1.to(device)).sample())
             if x1==2:
                
                return o
             x = self.transition((x))
             x1=(torch.distributions.Categorical(x.to(device)).sample()).item()



In [5]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Initial State
#         self.s_0 = nn.Parameter(torch.tensor([1.1, 1.1, 0.1], requires_grad=True))
        
#         # Transition Layers
#         self.t1 = nn.Linear(4, 32)
#         self.t2 = nn.Linear(32, 64)
#         self.t3 = nn.Linear(3, 3)
        
#         # Emission Layers
#         self.o1 = nn.Linear(4, 32)
#         self.o2 = nn.Linear(32, 64)
#         self.o3 = nn.Linear(3, 10)

#     def forward(self, x,t):
#         o = torch.zeros((t,10))
#         x = F.softmax((x))
#         for i in range(0,t):
# #             x = F.relu(self.t1(x))
# #             x = F.relu(self.t2(x))
#             x = F.softmax(self.t3(x))

# #             o_1= F.relu(self.o1(x))
# #             o_1 = F.relu(self.o2(o_1))
#             o_1 = F.softmax(self.o3(x))
#             o[i,:]=o_1
#         return x,o
    
#     def transition(self,x):
#         x = F.softmax((x))
# #         x = F.relu(self.t1(x))
# #         x = F.relu(self.t2(x))
#         x = F.softmax(self.t3(x))
#         return x
#     def Observation(self,x):
# #         o_1= F.relu(self.o1(x))
# #         o_1 = F.relu(self.o2(o_1))
#         o_1 = F.softmax(self.o3(x))
#         return o_1




In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [7]:

T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)
O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_mean1=torch.tensor([0.1,0.02,0.03,0.04]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)





print(T_matrix)

print(s_0)

tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [8]:
def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
#     o.append(Normal(O_matrix_mean[x],O_matrix_std[x]).sample().to(device))
    
    K=[2*i/(length) for i in range(1,length+1)]
    K=torch.tensor(K).to(device)
    o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        if x==0:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==1:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+torch.exp(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(3*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==2:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(torch.exp(O_matrix_mean1[x]*torch.tensor(t))+torch.sin(torch.pi*torch.tensor(t)/5))*torch.sin(6*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==3:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(torch.exp(O_matrix_mean1[x]*torch.tensor(t))+torch.sin(torch.pi*torch.tensor(t)/10))*torch.sin(9*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        t1=t
        if (t==50) :
            break
    
    

    return o,t,t1



In [9]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12768\2656443216.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12768\2656443216.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12

([tensor([ 1.1352,  2.2335,  2.8940,  4.6938, 11.2609,  6.1313,  3.2869,  2.1452,
          -0.6953,  1.6519]),
  tensor([ 3.7472,  1.1562,  3.7963,  6.7620, 10.2290,  6.5742,  2.0337,  0.2490,
           0.4323,  2.4395]),
  tensor([-0.6322,  0.9511,  3.4346,  3.3672, 10.9762,  3.5395,  4.0512, -1.2238,
           0.2366,  0.1018]),
  tensor([ 0.6384,  1.1853,  3.0366,  4.2066, 11.2059,  5.0518,  3.5234, -1.6548,
           2.1721,  3.6226]),
  tensor([ 1.2761, -0.8851,  5.0255,  5.8385, 11.8901,  2.0180,  2.8694, -0.5111,
           0.4427,  0.5586]),
  tensor([ 1.9893,  0.1223,  4.5035,  2.5305, 11.3644,  2.9642,  3.7220, -0.7311,
           0.9000,  1.4002]),
  tensor([ 2.4415, -0.0237,  3.8082,  2.7056, 11.3498,  3.9054,  3.9737, -0.3131,
           1.3004,  2.1114]),
  tensor([ 0.4676,  1.9416,  3.5761,  4.4782, 12.9611,  5.9147,  3.7973,  0.4859,
          -0.5175,  0.3691]),
  tensor([ 2.6857, -0.8660,  3.0596,  2.1322, 10.9037,  2.2847,  4.9404,  1.4079,
           0.0666,  1.

In [10]:
# def distribution_initalize(o):
#     s=torch.zeros(10)
#     for i in range(0,10):
#         s[i]=torch.sum(o==i)
        
#     return s#/torch.sum(s)
# def distribution_update(dist,o):

#     return dist + distribution_initalize(o)


In [11]:
# f=distribution_initalize(o[0])

def Expectation_diff(o_1):
    return torch.sum(o_1[0])


def abs_diff(e):
    k=0
    for i,e1 in enumerate(e):
        for j,e2 in enumerate(e):
            if j==i+1:
                k+=torch.abs(e2-e1)
    return k


In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time

start_time = time.time() 

for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        # batch_size=20
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.01)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred)
    
                    l1_lambda = 0.001  # adjust the L1 regularization parameter as needed
                    l1_reg = torch.tensor(0.).to(device)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1).to(device) 
                # #     print(dist)
                #     dist1=dist/torch.sum(dist)
                #     dist1=dist1.to(device)
                    f=0.0
                    # for j in range(0,t):
                    #         o_1= net.emission((torch.tensor([1.0,j]).to(device)))
                    #         e1=Expectation_diff(o_1)
    
                    #         o_1=net.emission((torch.tensor([2.0,j]).to(device)))
                    #         e2=Expectation_diff(o_1)
    
                    #         o_1= net.emission((torch.tensor([3.0,j]).to(device)))
                    #         e3=Expectation_diff(o_1)
    
    
                    #         o_1= net.emission((torch.tensor([4.0,j]).to(device)))
                    #         e4=Expectation_diff(o_1)
                    #         f+=abs_diff([e1,e2,e3,e4])
                   
    
    
                    #+ l1_reg*l1_lambda
                #     print(loss)
                    loss=loss#+ l1_reg*l1_lambda-0.001*f #+ 10*(torch.sum(t1*t2)/((torch.sum(t1**2)**0.5)*torch.sum(t2**2)**0.5) +torch.sum(t3*t2)/((torch.sum(t3**2)**0.5)*torch.sum(t2**2)**0.5)+torch.sum(t1*t3)/((torch.sum(t1**2)**0.5)*torch.sum(t3**2)**0.5) )
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                    # print statistics
            # print(f'[{trial + 1}] loss: {loss/t}')
    
            # print('Finished Training')
    
            torch.save(pred1, "data_normal\ " + str(trial)+" _ "+str(batch_size)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_normal\ " + str(trial)+" _ "+str(batch_size)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12768\2656443216.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12768\2656443216.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+(O_matrix_mean1[x]*torch.tensor(t))*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12